# ASDF from scratch
The tutorial for `pyasdf` at http://seismicdata.github.io/pyasdf/tutorial.html is a great starting point for general seismic analysis, i.e. assuming you have QuakeML and StationXML files ready to be assimilated into ASDF. My lab EQ data is all raw and messy, so I'll need to start out by creating these sub-structures.

## StationXML for the lab
I'll follow this ObsPy tutorial on __[creating a StationXML file from scratch](https://docs.obspy.org/tutorial/code_snippets/stationxml_file_from_scratch.html)__.

The lab sensor network is named the Glaser Lab Nano-Network (GLNN or GL) here, for local purposes only. This will fit into a file hierarchy of **Inventory > Network > Station > Channel**.

In [1]:
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL

Each of the hierarchy levels is an object that needs to be created. For now, I'm following the tutorial exactly rather than starting to incorporate real info about the GLNN.

In [2]:
inv = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id of whoever creates the file.
    source="ObsPy-Tutorial")

net = Network(
    # This is the network code according to the SEED standard.
    code="XX",
    # A list of stations. We'll add one later.
    stations=[],
    description="A test of stations.",
    # Start and end dates are optional.
    start_date=obspy.UTCDateTime(2016, 1, 2))

sta = Station(
    # This is the station code according to the SEED standard.
    code="ABC",
    latitude=1.0,
    longitude=2.0,
    elevation=345.0,
    creation_date=obspy.UTCDateTime(2016, 1, 2),
    site=Site(name="First station"))

cha = Channel(
    # This is the channel code according to the SEED standard.
    code="HHZ",
    # This is the location code according to the SEED standard.
    location_code="",
    # Note that these coordinates can differ from the station coordinates.
    latitude=1.0,
    longitude=2.0,
    elevation=345.0,
    depth=10.0,
    azimuth=0.0,
    dip=-90.0,
    sample_rate=200)

Next the tutorial pulls a station response file from the Nominal Response Library. This is a repository of instrument responses for commercial seismometers, but it's unlikely to be useful for lab sensors. I'll include it here for the sake of continuing to follow the tutorial.

In [3]:
# By default this accesses the NRL online. Offline copies of the NRL can
# also be used instead
nrl = NRL()
# The contents of the NRL can be explored interactively in a Python prompt,
# see API documentation of NRL submodule:
# http://docs.obspy.org/packages/obspy.clients.nrl.html
# Here we assume that the end point of data logger and sensor are already known:
response = nrl.get_response( # doctest: +SKIP
    sensor_keys=['Streckeisen', 'STS-1', '360 seconds'],
    datalogger_keys=['REF TEK', 'RT 130 & 130-SMA', '1', '200'])

Combine the objects we've created.

In [4]:
cha.response = response
sta.channels.append(cha)
net.stations.append(sta)
inv.networks.append(net)

Finally write it to a StationXML file. We also force a validation against the StationXML schema to ensure it produces a valid StationXML file. There's a note that "it's also possible to serialize to any of the other inventory output formats ObsPy supports." I'm not sure what that means.

In [5]:
inv.write("test_station.xml", format="stationxml", validate=True)

In [6]:
!ls ./

Learning_ASDF.ipynb  test_station.xml


## QuakeML from scratch
Next I need to figure out how to make a QuakeML file for a lab event. There is no tutorial I could find so we'll dig through the documentation. There's a quakeml package with a write function, `obspy.core.quakeml.writeQuakeML`, but it's doc has a big warning that it should only be used through the write function of an ObsPy `Catalog` object.

The __[Catalog class](http://docs.obspy.org/packages/autogen/obspy.core.event.catalog.Catalog.html#obspy.core.event.catalog.Catalog)__ is just a container for __[Event objects](http://docs.obspy.org/packages/autogen/obspy.core.event.event.Event.html#obspy.core.event.event.Event)__. `Event` objects start to have actual info in them, although a bunch of it also goes into sub-classes like `Pick`, `Origin`, `Magnitude`, etc. I'll go ahead and try to create a fully populated `Event` with dummy info.

In [1]:
import obspy
import obspy.core.event as ev

There seems to be a lot of value in properly used __[ResourceIdentifier](https://docs.obspy.org/packages/autogen/obspy.core.event.base.ResourceIdentifier.html#obspy.core.event.base.ResourceIdentifier)__s. It seems like most of the referencing will be auto-generated. I'll try starting with a `ResourceIdentifier` for an event and filling in from there.

In [7]:
event = ev.Event()
event.resource_id = ev.ResourceIdentifier(prefix='event', referred_object=event)
print(event, event.resource_id)

Event:	

 event/ed1c4557-361d-4087-ba7b-3cc5c2b03972


There's no laboraty event `EventType` so I'll use "other event" with "known" certainty.

In [21]:
event.event_type = ev.header.EventType(enum='other event')
event.event_type_certainty = ev.header.EventTypeCertainty(enum='known')
event.event_type = 'other ev'

ValueError: Setting attribute "event_type" failed. Value "other ev" could not be converted to type "Enum(["not existing", "not reported", "earthquake", ..., "rockslide", "meteorite", "volcanic eruption"])"

I noticed that the value actually stored for `event.event_type` was just the string 'other event', so I was curious if I could just assign a string instead of an `EventType` object. The error from trying to set 'other ev' shows that the `Event.event_type` parameter automatically invokes the `EventType` class and checks that the assigned string is in the enumerated list. A simple string assignment works with allowed value and produces exactly the same stored value as the object assignment I initially used.

The next parameter (alphabetically) is `Event.creation_info`. A `CreationInfo` object can contain an `agency_id`, an `agency_uri`, an `author`, an `author_uri`, a `creation_time`, and a `version`. The two uri fields both take (or create if called with just a string?) `ResourceIdentifier` objects. I'll fill in some dummy info for completeness of this exercise.

In [22]:
event.creation_info = ev.base.CreationInfo(author='JP', creation_time=obspy.core.UTCDateTime.utcnow())
print(event.creation_info)

CreationInfo(author='JP', creation_time=UTCDateTime(2018, 2, 6, 23, 4, 51, 234891))


Next is `Event.event_descriptions`, which can contain a list of `EventDescription` objects. Each of these contains a piece of descriptive text and an optional `EventDescriptionType` such as "earthquake name". The types are an extra bit of standardized metadata that could make filtering for events easier. I've tried adding a description but there is no automatic processing getting triggered. The assignment seems to be verbatim and the `event_descriptions` key doesn't show up as part of the `Event` dictionary. Something to come back to.

In [28]:
event.event_descriptions = [ev.event.EventDescription(text='a fake example event')]
print(event.event_descriptions)

[EventDescription(text='a fake example event')]


`Event.comments` is another list field, although `Comment` objects can have a `ResourceIdentifier` and `CreationInfo`. 

The remaining parameters for an `Event` are `picks`, `amplitudes`, `focal_mechanisms`, `origins`, `magnitudes`, and `station_magnitudes`. All of these parameters have their own classes and are able to contain quite a lot of inter-related information. Filling these in will make much more sense with sample data. For now, I have a very empty `Event` that should be enough to write a very empty QuakeML file to use in my ASDF trial.

In [31]:
catalog = ev.Catalog(creation_info=ev.base.CreationInfo(author='JP', creation_time=obspy.core.UTCDateTime.utcnow()))
catalog.resource_id = ev.ResourceIdentifier(prefix='catalog', referred_object=catalog)
catalog.append(event)

In [27]:
print(event.values())
event.event_descriptions

ValuesView(Event(resource_id=ResourceIdentifier(id="event/ed1c4557-361d-4087-ba7b-3cc5c2b03972"), event_type='other event', event_type_certainty='known', creation_info=CreationInfo(author='JP', creation_time=UTCDateTime(2018, 2, 6, 23, 4, 51, 234891))))


[EventDescription(text='a fake example event')]